In [1]:
library(IRdisplay)

display_html(
'<script>  
code_show=true; 
function code_toggle() {
  if (code_show){
    $(\'div.input\').hide();
  } else {
    $(\'div.input\').show();
  }
  code_show = !code_show
}  
$( document ).ready(code_toggle);
</script>
  <form action="javascript:code_toggle()">
    <input type="submit" value="Click here to toggle on/off the raw code.">
 </form>'
)

In [3]:
library(brms)
library(tidyverse)
library(parallel)

In [4]:
df <- read.table('MAX_neutral_early_late.txt',header = TRUE,sep = ",")
df <-filter(df,Phase=='early')
df$se = sqrt(df$var)

In [5]:
iterations <- 20000
chains <- 4
SCALE <- 1
ns <- iterations*chains/2

dataTable <- select(df,Subj,ROI,beta,se,TvsS,TRAIT,STATE)
colnames(dataTable)[5] <- "cond"
qVars = 'cond,STATE,TRAIT'
EOI = 'Intercept,cond'
EOIq <- strsplit(qVars, '\\,')[[1]]
if(!('Intercept' %in% EOIq)) EOIq <- c('Intercept', EOIq)
EOIq <- intersect(strsplit(EOI, '\\,')[[1]], EOIq)
print(paste0('Effect of interest: ',EOIq))


dataTable$Subj <- factor(dataTable$Subj)
dataTable$ROI <- factor(dataTable$ROI)

# number of ROIs
nR <- nlevels(dataTable$ROI)

print(paste0('Number of subjects: ',nlevels(dataTable$Subj)))
print(paste0('Number of ROIs: ',nlevels(dataTable$ROI)))
print(paste0('Number of cores available: ', detectCores(all.tests = FALSE, logical = TRUE)))

# number of sigfigs to show on the table
nfigs <- 4

head(dataTable)

[1] "Effect of interest: Intercept" "Effect of interest: cond"     
[1] "Number of subjects: 109"
[1] "Number of ROIs: 36"
[1] "Number of cores available: 16"


,Subj,ROI,beta,se,cond,TRAIT,STATE
,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,MAX101,ACC,-0.008450,0.03363034,-0.5,-1.164152,-1.011495
2,MAX101,Left AntDorsal Insula,-0.048800,0.02981610,-0.5,-1.164152,-1.011495
3,MAX101,Left AntVentral Insula,-0.129348,0.04368066,-0.5,-1.164152,-1.011495
4,MAX101,Left Anterior Putamen,-0.014730,0.04275512,-0.5,-1.164152,-1.011495
5,MAX101,Left Anterior Caudate,-0.072463,0.03974921,-0.5,-1.164152,-1.011495
6,MAX101,Left Anterior MCC,0.049450,0.04815600,-0.5,-1.164152,-1.011495


In [6]:
# Set nuber of cores to use. 
# To run each chain on a single core, set number of core to 4
print(getOption("mc.cores"))
options(mc.cores = parallel::detectCores())
print(getOption("mc.cores"))

NULL
[1] 16


## **1. BML**

## 1.1 Model 1

<center> Y $\sim$ 1 + cond + state + trait + (1 + cond | SUB) + (1 + cond + state + trait | ROI) </center>

$$Y_{s,r} \sim Student(\nu,\mu_{s,r},\sigma^{2}_{s,r})$$  
$$\mu_{s,r} = \alpha + \alpha_{SUB} + \alpha_{ROI} + (\beta_{cond} + \beta_{SUB_{cond}} + \beta_{ROI_{cond}})*cond + (\beta_{state} + \beta_{ROI_{state}})*state + (\beta_{trait} + \beta_{ROI_{trait}})*trait $$


Where,  
$\mu_{s,r}$: Mean ROI response in the given subject for a particular phase and block type.     
_cond_: Threat (0.5) vs. Safe (-0.5)  
_state_: Subject's state score  
_trait_: Subject's trait score  

## Priors

$\alpha \sim Student(3,0,10)$ [population effect]  
$\alpha_{SUB} \sim Student(\nu_{SUB},0,\sigma_{SUB})$ [Subject effect]    
$\alpha_{ROI} \sim Student(\nu_{ROI},0,\sigma_{ROI})$ [ROI effect]  

$\beta_{cond} \sim Student(3,0,10)$ [population effect: cond]  
$\beta_{state} \sim Student(3,0,10)$ [population effect: state]    
$\beta_{trait} \sim Student(3,0,10)$ [population effect: trait]  

$\begin{bmatrix}
  \alpha_{ROI} \\
  \beta_{ROI_{cond}} \\
  \beta_{ROI_{state}} \\
  \beta_{ROI_{trait}}
\end{bmatrix} \sim MVNormal(\begin{bmatrix} 0 \\ 0 \\ 0 \\ 0 \end{bmatrix},S)$

$ S = \begin{bmatrix} \sigma_{\alpha_{ROI}} & & & \\ & \sigma_{\beta_{ROI_{cond}}} & & \\ & & \sigma_{\beta_{ROI_{state}}} & \\ & & & \sigma_{\beta_{ROI_{trait}}} \end{bmatrix} R \begin{bmatrix} \sigma_{\alpha_{ROI}} & & & \\ & \sigma_{\beta_{ROI_{cond}}} & & \\ & & \sigma_{\beta_{ROI_{state}}} & \\ & & & \sigma_{\beta_{ROI_{trait}}} \end{bmatrix}$

$\begin{bmatrix}
  \alpha_{SUB} \\
  \beta_{SUB_{cond}}
\end{bmatrix} \sim MVNormal(\begin{bmatrix} 0 \\ 0 \end{bmatrix},S)$

$ S = \begin{bmatrix} \sigma_{\alpha_{SUB}} & \\ & \sigma_{\beta_{SUB_{cond}}} \end{bmatrix} R \begin{bmatrix} \sigma_{\alpha_{SUB}} & \\ & \sigma_{\beta_{SUB_{cond}}} \end{bmatrix}$


$\sigma_{SUB} \sim HalfStudent(3,0,10)$ [prior stddev for Subject effect]    
$\sigma_{ROI} \sim HalfStudent(3,0,10)$ [prior stddev for ROI effect]    
$\nu_{SUB} \sim \Gamma(3.325,0.1)$ [Subject effect hyperprior]    
$\nu_{ROI} \sim \Gamma(3.325,0.1)$ [ROI effect hyperprior]  

$\sigma_{\alpha} \sim HalfStudent(3,0,10)$ [prior stddev among intercepts]  
$\sigma_{\beta} \sim HalfStudent(3,0,10)$ [prior stddev among slopes]

$R \sim LKJcorr(2)$ [prior of correlation matix]


In [10]:
mod = '1 + cond + STATE + TRAIT'
modelForm = paste('beta | se(se) ~',
                  mod,'+ (1 + cond | gr(Subj, dist= "student")) +   
                         (',mod,'| gr(ROI, dist="student"))')
priorRBA <- get_prior(formula = modelForm,data=dataTable,family = 'student')
priorRBA$prior[5] <- "lkj(2)"
priorRBA$prior[8:9] <- "gamma(3.325,0.1)"
priorRBA$prior[11] <- "gamma(3.325,0.1)"
print(priorRBA)

                 prior     class      coef group resp dpar nlpar bound
1                              b                                      
2                              b      cond                            
3                              b     STATE                            
4                              b     TRAIT                            
5               lkj(2)       cor                                      
6                            cor             ROI                      
7                            cor            Subj                      
8     gamma(3.325,0.1)        df             ROI                      
9     gamma(3.325,0.1)        df            Subj                      
10 student_t(3, 0, 10) Intercept                                      
11    gamma(3.325,0.1)        nu                                      
12 student_t(3, 0, 10)        sd                                      
13                            sd             ROI                      
14    

In [ ]:
mod1 <- brm(modelForm,
          data=dataTable,
          chains = chains,
          family = 'student',
          prior = priorRBA,
          inits=0, iter=iterations, 
          control = list(adapt_delta = 0.99, max_treedepth = 15))

Compiling the C++ model

Start sampling



## 1.2 Model 2

<center> Y $\sim$ 1 + cond + state + trait + (1 | SUB) + (1 + cond + state + trait | ROI) </center>

$$Y_{s,r} \sim Student(\nu,\mu_{s,r},\sigma^{2}_{s,r})$$  
$$\mu_{s,r} = \alpha + \alpha_{SUB} + \alpha_{ROI} + (\beta_{cond} + \beta_{SUB_{cond}} + \beta_{ROI_{cond}})*cond + (\beta_{state} + \beta_{ROI_{state}})*state + (\beta_{trait} + \beta_{ROI_{trait}})*trait $$


Where,  
$\mu_{s,r}$: Mean ROI response in the given subject for a particular phase and block type.     
_cond_: Threat (0.5) vs. Safe (-0.5)  
_state_: Subject's state score  
_trait_: Subject's trait score  

## Priors

$\alpha \sim Student(3,0,10)$ [population effect]  
$\alpha_{SUB} \sim Student(\nu_{SUB},0,\sigma_{SUB})$ [Subject effect]    
$\alpha_{ROI} \sim Student(\nu_{ROI},0,\sigma_{ROI})$ [ROI effect]  

$\beta_{cond} \sim Student(3,0,10)$ [population effect: cond]  
$\beta_{state} \sim Student(3,0,10)$ [population effect: state]    
$\beta_{trait} \sim Student(3,0,10)$ [population effect: trait]  

$\begin{bmatrix}
  \alpha_{ROI} \\
  \beta_{ROI_{cond}} \\
  \beta_{ROI_{state}} \\
  \beta_{ROI_{trait}}
\end{bmatrix} \sim MVNormal(\begin{bmatrix} 0 \\ 0 \\ 0 \\ 0 \end{bmatrix},S)$

$ S = \begin{bmatrix} \sigma_{\alpha_{ROI}} & & & \\ & \sigma_{\beta_{ROI_{cond}}} & & \\ & & \sigma_{\beta_{ROI_{state}}} & \\ & & & \sigma_{\beta_{ROI_{trait}}} \end{bmatrix} R \begin{bmatrix} \sigma_{\alpha_{ROI}} & & & \\ & \sigma_{\beta_{ROI_{cond}}} & & \\ & & \sigma_{\beta_{ROI_{state}}} & \\ & & & \sigma_{\beta_{ROI_{trait}}} \end{bmatrix}$


$\sigma_{SUB} \sim HalfStudent(3,0,10)$ [prior stddev for Subject effect]    
$\sigma_{ROI} \sim HalfStudent(3,0,10)$ [prior stddev for ROI effect]    
$\nu_{SUB} \sim \Gamma(3.325,0.1)$ [Subject effect hyperprior]    
$\nu_{ROI} \sim \Gamma(3.325,0.1)$ [ROI effect hyperprior]  

$\sigma_{\alpha} \sim HalfStudent(3,0,10)$ [prior stddev among intercepts]  
$\sigma_{\beta} \sim HalfStudent(3,0,10)$ [prior stddev among slopes]

$R \sim LKJcorr(2)$ [prior of correlation matix]


In [11]:
mod = '1 + cond + STATE + TRAIT'
modelForm = paste('beta | se(se) ~',
                  mod,'+ (1 | gr(Subj, dist= "student")) +   
                         (',mod,'| gr(ROI, dist="student"))')
priorRBA <- get_prior(formula = modelForm,data=dataTable,family = 'student')
priorRBA$prior[5] <- "lkj(2)"
priorRBA$prior[7:8] <- "gamma(3.325,0.1)"
priorRBA$prior[10] <- "gamma(3.325,0.1)"
priorRBA

,prior,class,coef,group,resp,dpar,nlpar,bound
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,,b,,,,,,
2,,b,cond,,,,,
3,,b,STATE,,,,,
4,,b,TRAIT,,,,,
5,lkj(2),cor,,,,,,
6,,cor,,ROI,,,,
7,"gamma(3.325,0.1)",df,,ROI,,,,
8,"gamma(3.325,0.1)",df,,Subj,,,,
9,"student_t(3, 0, 10)",Intercept,,,,,,


In [ ]:
mod2 <- brm(modelForm,
          data=dataTable,
          chains = chains,
          family = 'student',
          prior = priorRBA,
          inits=0, iter=iterations, 
          control = list(adapt_delta = 0.99, max_treedepth = 15))

## Leave one out cross-validation & model comparison

In [1]:
mod1 <- add_criterion(mod1,c("loo","waic","bayes_R2"))
mod2 <- qqadd_criterion(mod2,c("loo","waic","bayes_R2"))

ERROR: Error in add_criterion(mod1, c("loo", "waic", "bayes_R2")): could not find function "add_criterion"


In [ ]:
# Compare both models
comparison_table <- loo_compare(mod1,mod2)
comparison_table

## Goodness of fit

In [ ]:
R2_table <- data.frame(sapply(list(mod1,mod2), function(x) round(median(x$criteria$bayes_R2)*100, 2)),row.names = c('mod1','mod2'))
names(R2_table) <- "Bayes R2"
R2_table